<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [ ]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [ ]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [ ]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [ ]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [ ]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True


In [ ]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [ ]:
inputs.Age[:10]

,Age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,NaN
6,54.0
7,2.0
8,27.0
9,14.0


In [ ]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
model.fit(X_train,y_train)

GaussianNB()

In [ ]:
model.score(X_test,y_test)

0.7761194029850746

In [ ]:
X_test[0:10]

,Pclass,Age,Fare,female
470,3,29.699118,7.2500,False
530,2,2.000000,26.0000,True
344,2,36.000000,13.0000,False
790,3,29.699118,7.7500,False
218,1,32.000000,76.2917,True
300,3,29.699118,7.7500,True
371,3,18.000000,6.4958,False
57,3,28.500000,7.2292,False
621,1,42.000000,52.5542,False
755,2,0.670000,14.5000,False


In [ ]:
y_test[0:10]

,Survived
470,0
530,1
344,0
790,0
218,1
300,1
371,0
57,0
621,1
755,1


In [ ]:
model.predict(X_test[0:10])

array([0, 1, 0, 0, 1, 1, 0, 0, 0, 0])

In [ ]:
model.predict_proba(X_test[:10])

array([[0.96734359, 0.03265641],
       [0.14849188, 0.85150812],
       [0.93375588, 0.06624412],
       [0.96743288, 0.03256712],
       [0.02793848, 0.97206152],
       [0.439123  , 0.560877  ],
       [0.9609544 , 0.0390456 ],
       [0.96698967, 0.03301033],
       [0.70841336, 0.29158664],
       [0.86646043, 0.13353957]])

**Calculate the score using cross validation**

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.856     , 0.752     , 0.768     , 0.73387097, 0.79032258])

Problem Statement: Email Phishing Detection

Goal: Predict whether an email is phishing (fraudulent) or legitimate based on its content and metadata.

Dataset features might include:
<ol>
<li>email_length → number of characters in the email


<li>num_links → number of hyperlinks in the email

<li>num_special_chars → number of suspicious characters (e.g., $, %, @)

<li>contains_login_request → whether the email asks for login details (Yes/No)

<li>sender_domain → domain of the sender (categorical)

Target: is_phishing → 1 for phishing, 0 for legitimate

| Email\_ID | Subject                                      | Contains\_Link | Contains\_Attachment | Urgent\_Words | From\_Trusted\_Domain | Label    |
| --------- | -------------------------------------------- | -------------- | -------------------- | ------------- | --------------------- | -------- |
| 1         | "Urgent: Verify your bank account"           | Yes            | No                   | Yes           | No                    | Phishing |
| 2         | "Meeting schedule for next week"             | No             | Yes                  | No            | Yes                   | Legit    |
| 3         | "Claim your lottery prize now"               | Yes            | No                   | Yes           | No                    | Phishing |
| 4         | "Invoice attached for your recent purchase"  | No             | Yes                  | No            | Yes                   | Legit    |
| 5         | "Security alert: Unusual login detected"     | Yes            | No                   | Yes           | No                    | Phishing |
| 6         | "Team lunch invitation"                      | No             | No                   | No            | Yes                   | Legit    |
| 7         | "Update your payment information"            | Yes            | Yes                  | Yes           | No                    | Phishing |
| 8         | "Your order has been shipped"                | No             | No                   | No            | Yes                   | Legit    |
| 9         | "Reset your password immediately"            | Yes            | No                   | Yes           | No                    | Phishing |
| 10        | "Happy Birthday from all of us!"             | No             | No                   | No            | Yes                   | Legit    |
| 11        | "Verify your email to avoid suspension"      | Yes            | No                   | Yes           | No                    | Phishing |
| 12        | "Monthly performance report attached"        | No             | Yes                  | No            | Yes                   | Legit    |
| 13        | "Get rich quick investment opportunity"      | Yes            | No                   | Yes           | No                    | Phishing |
| 14        | "Reminder: Project deadline next Monday"     | No             | No                   | No            | Yes                   | Legit    |
| 15        | "Your account will be closed unless you act" | Yes            | No                   | Yes           | No                    | Phishing |


<h2> Sample Prediction Input

After fitting your Naive Bayes model, predict

| Subject                            | Contains\_Link | Contains\_Attachment | Urgent\_Words | From\_Trusted\_Domain |
| ---------------------------------- | -------------- | -------------------- | ------------- | --------------------- |
| "Update your password to continue" | 1              | 0                    | 1             | 0                     |
